In [1]:
# BIG GOTCHA WARNING - Must figure out why column for year/FSM/SEN/gender
# is not being created for some schools, likely to be an issue in dataset (e.g. missing data)

from pathlib import Path

import pandas as pd


In [ ]:
# Read in the RAG dataset
rag_df = pd.read_csv(
    "../data/real/standard_area_aggregate_scores_rag.csv",
)
# Replace specific values in 'variable_lab' and 'description' columns for clarity
rag_df["variable_lab"] = rag_df["variable_lab"].replace(
    {
        "Taking to staff about feelings": "Talking to staff about feelings",
        "Taking at home about feelings": "Talking at home about feelings",
        "Taking to peers about feelings": "Talking to peers about feelings",
    },
)

# Update 'group' column for specific measure_lab rows
rag_df.loc[rag_df["variable"].isin(["staff_talk_score"]), ["variable_lab", "description", "group"]] = [
    "Talking to staff about feelings",
    "How positively/negatively young people feel about talking with staff about feeling down",
    "staff_talk",
]

rag_df.loc[rag_df["variable"].isin(["peer_talk_score"]), ["variable_lab", "description", "group"]] = [
    "Talking to peers about feelings",
    "How positively/negatively young people feel about talking with peers about feeling down",
    "peer_talk",
]

rag_df.loc[rag_df["variable"].isin(["home_talk_score"]), ["variable_lab", "description", "group"]] = [
    "Talking at home about feelings",
    "How positively/negatively young people feel about talking at home about feeling down",
    "home_talk",
]

# Remove the 'talk_score' rows, since they are redundant with the split
rag_df = rag_df[rag_df["variable"] != "talk_score"]

# Display the first few rows to confirm changes
print(rag_df.head())
 
rag_df.head()

In [ ]:
RESULT_NUMBER_THRESHOLD = 10

# Select relevant columns
rag_columns = [
    "variable_lab",
    "rag",
    "school_lab",
    "year_group_lab",
    "gender_lab",
    "fsm_lab",
    "sen_lab",
    "count",
]
filtered_rag_df = rag_df[rag_columns]

filtered_rag_df["rag"] = filtered_rag_df["rag"].replace(
    {
        "average": "Average",
        "above": "Above average",
        "below": "Below average",
    },
)

# **1. Convert 'count' column to numeric**
filtered_rag_df["count"] = (
    pd.to_numeric(filtered_rag_df["count"], errors="coerce").fillna(0).astype(int)
)
# Create dictionary to store dfs for each school
school_dfs: dict[str, pd.DataFrame] = {}

# Get the unique school names
schools = filtered_rag_df["school_lab"].unique()

# Iterate over each school and create a DataFrame for "All pupils"
for school in schools:
    # Filter by school
    school_df = filtered_rag_df[filtered_rag_df["school_lab"] == school]

    # Filter by "All pupils" (where all grouping variables are set to "All")
    all_pupils_df = school_df[
        (school_df["year_group_lab"] == "All")
        & (school_df["gender_lab"] == "All")
        & (school_df["fsm_lab"] == "All")
        & (school_df["sen_lab"] == "All")
    ].copy()

    # **2. Ensure 'count' is numeric in all_pupils_df as well**
    all_pupils_df["count"] = (
        pd.to_numeric(all_pupils_df["count"], errors="coerce").fillna(0).astype(int)
    )
    # Insert "n<10" into the 'rag' column where 'count' is less than 10
    all_pupils_df.loc[all_pupils_df["count"] < RESULT_NUMBER_THRESHOLD, "rag"] = f"n<{RESULT_NUMBER_THRESHOLD}"

    # Select relevant columns: 'variable_lab', 'rag', 'school_lab', and add 'All pupils' as the group
    all_pupils_df = all_pupils_df[["variable_lab", "rag", "school_lab"]]
    all_pupils_df["group"] = "All pupils"

    # Store the DataFrame in the school_dfs dictionary
    school_dfs[f"{school}_all_pupils"] = all_pupils_df

# To view the DataFrame for 'School B' for "All pupils"
school_b_all_pupils_df = school_dfs.get("School B_all_pupils")
# Display the first few rows of the DataFrame for 'School B'


# Save each DataFrame to a CSV file
for school in schools:
    # Ensure the directory exists
    output_dir = Path(f"outputs/{school}/rag_ratings")
    output_dir.mkdir(parents=True, exist_ok=True)

    # Save the DataFrame to a CSV file
    school_dfs[f"{school}_all_pupils"].to_csv(output_dir / f"rag_all_pupils_{school}.csv", index=False)


In [4]:
# By year group
# Create a year group pivot DataFrame for each school
for school in schools:
    school_df = filtered_rag_df[filtered_rag_df["school_lab"] == school][rag_columns]

    # Filter by year groups
    year_group_rag: pd.DataFrame = school_df[
        school_df["year_group_lab"].isin(["Year 8", "Year 10"])
    ]

    # Pivot the table for Year 8 and Year 10 RAG summaries
    year_group_pivot = year_group_rag.pivot_table(
        index=["school_lab", "variable_lab"],
        columns="year_group_lab",
        values="rag",
        aggfunc="first",
    ).reset_index()

    # Store the pivoted DataFrame in the school_dfs dictionary
    school_dfs[f"{school}_year_group"] = year_group_pivot

# Save each DataFrame to a CSV file
for school in schools:
    # Ensure the directory exists
    output_dir = Path(f"outputs/{school}/rag_ratings")
    output_dir.mkdir(parents=True, exist_ok=True)

    # Save the DataFrame to a CSV file
    school_dfs[f"{school}_year_group"].to_csv(
        output_dir / f"rag_year_group_{school}.csv", index=False,
    )





In [5]:
# By FSM status
for school in schools:
    school_df = filtered_rag_df[filtered_rag_df["school_lab"] == school][rag_columns]

    # Filter by FSM
    fsm_pivot_rag: pd.DataFrame = school_df[
        school_df["fsm_lab"].isin(["FSM", "Non-FSM"])
    ]

    # Pivot the table for SEN status
    fsm_pivot = fsm_pivot_rag.pivot_table(
        index=["school_lab", "variable_lab"],
        columns="fsm_lab",
        values="rag",
        aggfunc="first",
    ).reset_index()

    # Store the pivoted DataFrame in the school_dfs dictionary
    school_dfs[f"{school}_fsm"] = fsm_pivot

# Save each DataFrame to a CSV file
for school in schools:
    # Ensure the directory exists
    output_dir = Path(f"outputs/{school}/rag_ratings")
    output_dir.mkdir(parents=True, exist_ok=True)

    # Save the DataFrame to a CSV file
    school_dfs[f"{school}_fsm"].to_csv(
        output_dir / f"rag_fsm_{school}.csv", index=False,
    )


In [6]:
# By Gender
for school in schools:
    school_df = filtered_rag_df[filtered_rag_df["school_lab"] == school][rag_columns]

    # Filter by gender
    gender_rag: pd.DataFrame = school_df[school_df["gender_lab"].isin(["Boy", "Girl"])]

    # Pivot
    gender_pivot = gender_rag.pivot_table(
        index=["school_lab", "variable_lab"],
        columns="gender_lab",
        values="rag",
        aggfunc="first",
    ).reset_index()

    # Store the pivoted DataFrame in the school_dfs dictionary
    school_dfs[f"{school}_gender"] = gender_pivot

# Save each DataFrame to a CSV file
for school in schools:
    # Ensure the directory exists
    output_dir = Path(f"outputs/{school}/rag_ratings")
    output_dir.mkdir(parents=True, exist_ok=True)

    # Save the DataFrame to a CSV file
    school_dfs[f"{school}_gender"].to_csv(
        output_dir / f"rag_gender_{school}.csv", index=False,
    )


In [7]:
# By SEN status

# Create a year group pivot DataFrame for each school
for school in schools:
    school_df = filtered_rag_df[filtered_rag_df["school_lab"] == school][rag_columns]

    # Filter by year groups
    sen_rag: pd.DataFrame = school_df[school_df["sen_lab"].isin(["SEN", "Non-SEN"])]

    # Pivot the table for SEN status
    sen_pivot = sen_rag.pivot_table(
        index=["school_lab", "variable_lab"],
        columns="sen_lab",
        values="rag",
        aggfunc="first",
    ).reset_index()

    # Store the pivoted DataFrame in the school_dfs dictionary
    school_dfs[f"{school}_sen"] = sen_pivot

# Save each DataFrame to a CSV file
for school in schools:
    # Ensure the directory exists
    output_dir = Path(f"outputs/{school}/rag_ratings")
    output_dir.mkdir(parents=True, exist_ok=True)

    # Save the DataFrame to a CSV file
    school_dfs[f"{school}_sen"].to_csv(
        output_dir / f"rag_sen_{school}.csv", index=False,
    )
